### 1. Start by retrieving the names of the AOMs present in transport.data.gouv.fr

In [ ]:
import requests

# API URL
url = "https://transport.data.gouv.fr/api/stats"

# Send a GET request to the API
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()

    # Extract 'nom' field from each feature
    noms = [feature['properties']['nom'] for feature in data['features']]
    dataset_count = [feature['properties']['dataset_count'] for feature in data['features']]
    
    #"dataset_formats":{"gtfs":1,"gtfs_rt":4,"netex":1,"siri":2}
    dataset_formats = [feature['properties']['dataset_formats'] for feature in data['features']]
    

    # Print the extracted 'nom' and 'dataset_count' fields
    for nom, datas,format in zip(noms, dataset_count, dataset_formats):
        print(f"{nom} : {datas} + {format}")
        
else:
    print("Failed to fetch data from the API")


### 2. Send the list into a .csv file

In [4]:
#push to a .csv file
import csv
with open('filled_aoms.csv', 'w', newline='') as csvfile:
    fieldnames = ['nom', 'dataset_count','details']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for nom,nb_data,format in zip(noms, dataset_count, dataset_formats):
        writer.writerow({'nom': nom, 'dataset_count': nb_data,'details':format})
        


### 3.Retrieve the list of all AOMs

[Source](https://www.data.gouv.fr/fr/datasets/liste-et-composition-des-autorites-organisatrices-de-la-mobilite-aom/)

In [6]:
import pandas as pd

# Read the data from liste_aoms.csv
liste_aoms = pd.read_csv('liste_aoms.csv')



### 4. Substract the AOMs present in transport.data.gouv.fr from the list of all AOMs

In order to get the list of AOMs who have not published any dataset yet 😉

In [8]:
# Read the data from data.csv
data = pd.read_csv('filled_aoms.csv')

# Get the list of "nom" values from data.csv
nom_values = data['nom'].tolist()

# Exclude rows where "Nom de l’AOM" equals one of the "nom" values
filtered_aoms = liste_aoms[~liste_aoms['Nom de l’AOM'].isin(nom_values)]

# Print the filtered dataframe
print(filtered_aoms)

#send data to a .csv called empty_aoms.csv
filtered_aoms.to_csv('empty_aoms.csv', index=False)


      N° SIREN\nAOM                                       Nom de l’AOM  \
58        248400293                           CA des Sorgues du Comtat   
95        200023778            CA du Pays de Saint-Gilles-Croix-de-Vie   
166       200070233                              CA Terres de Montaigu   
177       247104094                           CC  Sud Côte Chalonnaise   
178       241300375              CC  Vallée des Baux-Alpilles (CC VBA)   
...             ...                                                ...   
1164      231300021  Région Provence-Alpes-Côte d'Azur (CC Méditerr...   
1165      231300021  Région Provence-Alpes-Côte d'Azur (CC Aygues-O...   
1166      231300021  Région Provence-Alpes-Côte d'Azur (CC Enclave ...   
1167      231300021  Région Provence-Alpes-Côte d'Azur (CC Pays d'A...   
1168      231300021  Région Provence-Alpes-Côte d'Azur (CC Ventoux ...   

     Commune principale\nDe l’AOM  N° SIREN\nGroupement  \
58                        Sorgues             248400

## Retrieve coordinates of empty AOMs

In [ ]:
import requests

# API URL
url = "https://transport.data.gouv.fr/api/aoms/geojson"

# Send a GET request to the API
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()

    # Extract 'properties' and 'geometry' fields from each feature
    entities = []
    for feature in data['features']:
        entity = {
            'name': feature['properties']['nom'],
            'coordinates': feature['geometry']['coordinates']
        }
        entities.append(entity)

    # Print the extracted 'name' and 'coordinates' fields
    for entity in entities:
        print(f"Name: {entity['name']}")
        print(f"Coordinates: {entity['coordinates']}")
        print()
else:
    print("Failed to fetch data from the API")


### Retrieve coordinates of empty AOMs

In [12]:
import pandas as pd

# Read the data from empty_aoms.csv
empty_aoms = pd.read_csv('empty_aoms.csv')

# Create a dictionary to store the coordinates
coordinates_dict = {}

# Iterate over the entities and populate the coordinates dictionary
for entity in entities:
    coordinates_dict[entity['name']] = entity.get('coordinates')

# Create a list to store the coordinates for each row in the DataFrame
coordinates = []

# Iterate over the rows in the DataFrame and fetch coordinates from the dictionary
for name in empty_aoms['Nom de l’AOM']:
    coordinates.append(coordinates_dict.get(name))

# Add the coordinates as a new column to empty_aoms
empty_aoms['Coordinates'] = coordinates

# Save the updated dataframe to empty_aoms.csv
empty_aoms.to_csv('empty_aoms.csv', index=False)


In [33]:
import csv
import geojson

# Define the input and output file paths
input_csv_file = 'empty_aoms.csv'
output_geojson_file = 'output_file.geojson'

# Create a GeoJSON FeatureCollection
features = []

# Open the CSV file and read its contents
with open(input_csv_file, 'r') as csv_file:
    csv_reader = csv.DictReader(csv_file)

    # Iterate over each row in the CSV file
    for row in csv_reader:
        # Extract required fields
        name = row["Nom de l’AOM"]
        coordinates_str = row['Coordinates']

        # Parse coordinates string into a list
        try:
            # Extract the inner list of coordinates
            coordinates = eval(coordinates_str)[0]
        except (SyntaxError, IndexError) as e:
            print(f"Error extracting coordinates for '{name}': {e}")
            continue

        # Create a GeoJSON Polygon
        polygon = geojson.Polygon([coordinates])

        # Create a GeoJSON Feature
        feature = geojson.Feature(geometry=polygon, properties={'name': name})

        # Add the Feature to the FeatureCollection
        features.append(feature)

# Create a FeatureCollection
feature_collection = geojson.FeatureCollection(features)

# Write the FeatureCollection to a GeoJSON file
with open(output_geojson_file, 'w') as geojson_file:
    geojson.dump(feature_collection, geojson_file, indent=2)

print("GeoJSON file created successfully!")


Error extracting coordinates for 'Collectivité d’Outre-Mer de SAINT-MARTIN': invalid syntax (<string>, line 0)
Error extracting coordinates for 'ILE D'YEU': invalid syntax (<string>, line 0)
Error extracting coordinates for 'Île-de-France Mobilités ': invalid syntax (<string>, line 0)
Error extracting coordinates for 'SIVOTU (nouvelle dénomination le 24/02/2010:AGGLOBUS)': invalid syntax (<string>, line 0)
GeoJSON file created successfully!
